## Data Checks

- Schema checks:
    - Making sure that only the columns that are expected are provided.
    - Making sure the data types are correct: `str`/`object`, `int`, `float32`, `float64`, `datetime`.
- Datum checks:
    - Looking for missing values
    - Ensuring that expected value ranges are correct
- Statistical checks:
    - Visual check of data distributions.
    - Correlations between columns.
    - Statistical distribution checks.

## Schema Checks

Over time, you've decided that the necessarily columns of data that you'll need for production use include the following:

- 

## Datum Checks

## Statistical Checks